# VGG-16 : Optimization

In [97]:
import math
import numpy as np
import tensorflow as tf
import keras
from keras import applications, Model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.metrics import make_scorer
from keras.utils.np_utils import to_categorical
import random
import sklearn
import pandas as pd
import warnings

warnings.simplefilter('ignore')

In [98]:
train_data_dir = 'Data/Training_Data'
validation_data_dir = 'Data/Validation_Data'
test_data_dir= 'Data/Test_Data'

training_features_file = 'Features/training_features_VGG16.npy'
validation_features_file = 'Features/validation_features_VGG16.npy'
top_weights_file = 'Weights/weights_VGG16.h5'
model_file = 'Models/model_vgg16.h5'

train_labels_file = 'Labels/training_labels.npy'
validation_labels_file = 'Labels/validation_labels.npy'
test_labels_file = 'Labels/test_labels.npy'

img_width, img_height = 224, 224
NB_CLASSES = 11

In [99]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

In [100]:
print('Training Data : ' + str(len(train_labels)) + ' Images')
print('Validation Data : ' + str(len(validation_labels)) + ' Images')
print('Test Data : ' + str(len(test_labels)) + ' Images')

Training Data : 307 Images
Validation Data : 74 Images
Test Data : 18 Images


## Converting images to feature vectors using weights from ImageNet

In [101]:
def images_to_feature_vectors(model, directory, batch_size, steps):
    
    datagen = ImageDataGenerator()
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) # Keep the data in the same order
    
    features = model.predict_generator(generator, steps, verbose=1) 
    
    return features

In [102]:
# Batch size has to be a multiple of the number of images  to keep our vectors consistents
training_batch_size = 1 # batch size for feature pre-training
validation_batch_size = 1 # batch size for feature pre-training

base_model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3)) #VGG16 trained on imagenet
training_features = images_to_feature_vectors(base_model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(base_model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

Found 307 images belonging to 11 classes.
307/307 [==============================] - 55s 179ms/step
Found 74 images belonging to 11 classes.
74/74 [==============================] - 12s 159ms/step


In [103]:
with open(training_features_file, 'wb') as file:
        np.save(file, training_features, allow_pickle = False)
with open(validation_features_file, 'wb') as file:
        np.save(file, validation_features, allow_pickle = False)

## Randomized Search CV training

In [104]:
def create_model(lr, decay, nn1, nn2, nn3, input_shape, output_shape):
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''
    
    opt = keras.optimizers.Adam(lr=lr, decay=decay)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu'))
    model.add(BatchNormalization())
            
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [105]:
model = KerasClassifier(build_fn=create_model, epochs=16, verbose=1)

In [106]:
# Learning rate values
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# Number of neurons per layer
nn1=[4096,2048,1024]
nn2=[2048,1024,512]
nn3=[1000,500,200]

batch_size=[2048,1024,512]

In [107]:
train_data = np.load(open(training_features_file, 'rb'))
#train_data = train_data.reshape(train_data.shape[0],-1)
validation_data = np.load(open(validation_features_file, 'rb'))
#validation_data = validation_data.reshape(validation_data.shape[0],-1)
    
train_labels_onehot = to_categorical(train_labels,NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels,NB_CLASSES)  #One Hot Encoder

In [108]:
# dictionary summary
param_grid = dict(
                    lr=lr, decay=decay, nn1=nn1, nn2=nn2, nn3=nn3,
                    batch_size=batch_size,
                    input_shape=train_data.shape[1:], output_shape = (NB_CLASSES,)
                 )


grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=20,  n_iter=10, n_jobs=1)

In [60]:
grid_result = grid.fit(train_data, train_labels_onehot)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] output_shape=101, nn3=500, nn2=512, nn1=4096, lr=0.001, input_shape=7, decay=1e-06, batch_size=2048 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/16
204/204 [==============================] - 5s 27ms/step - loss: 5.4665 - acc: 0.0098
Epoch 2/16
204/204 [==============================] - 2s 7ms/step - loss: 0.1609 - acc: 1.0000
Epoch 3/16
204/204 [==============================] - 2s 8ms/step - loss: 0.0134 - acc: 1.0000
Epoch 4/16
204/204 [==============================] - 2s 8ms/step - loss: 0.0060 - acc: 1.0000
Epoch 5/16
204/204 [==============================] - 2s 8ms/step - loss: 0.0037 - acc: 1.0000
Epoch 6/16
204/204 [==============================] - 2s 8ms/step - loss: 0.0026 - acc: 1.0000
Epoch 7/16
204/204 [==============================] - 2s 7ms/step - loss: 0.0019 - acc: 1.0000
Epoch 8/16
204/204 [==============================] - 2s 8ms/step - loss: 0.0015 - acc: 1.0000
Epoch 9/16
204/204 [==============================] - 2s 8ms/step - loss: 0.0012 - acc: 1.0000
Epoch 10/16
204/204 [==============================] - 2s 8ms/step - loss: 9.6188e-04 - acc: 1.0000
Epoch 11/16
204/204 [=======================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.8s remaining:    0.0s


Epoch 1/16
205/205 [==============================] - 8s 38ms/step - loss: 5.3803 - acc: 0.0195
Epoch 2/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0867 - acc: 1.0000
Epoch 3/16
205/205 [==============================] - 2s 7ms/step - loss: 0.0107 - acc: 1.0000
Epoch 4/16
205/205 [==============================] - 2s 7ms/step - loss: 0.0043 - acc: 1.0000
Epoch 5/16
205/205 [==============================] - 2s 7ms/step - loss: 0.0023 - acc: 1.0000
Epoch 6/16
205/205 [==============================] - 2s 7ms/step - loss: 0.0015 - acc: 1.0000
Epoch 7/16
205/205 [==============================] - 2s 7ms/step - loss: 0.0011 - acc: 1.0000
Epoch 8/16
205/205 [==============================] - 2s 7ms/step - loss: 8.0083e-04 - acc: 1.0000
Epoch 9/16
205/205 [==============================] - 2s 7ms/step - loss: 6.3142e-04 - acc: 1.0000
Epoch 10/16
205/205 [==============================] - 2s 7ms/step - loss: 5.1540e-04 - acc: 1.0000
Epoch 11/16
205/205 [===============

...

Epoch 1/16
205/205 [==============================] - 9s 45ms/step - loss: 5.1331 - acc: 0.0146
Epoch 2/16
205/205 [==============================] - 1s 4ms/step - loss: 0.7881 - acc: 0.9902
Epoch 3/16
205/205 [==============================] - 1s 4ms/step - loss: 0.1757 - acc: 1.0000
Epoch 4/16
205/205 [==============================] - 1s 4ms/step - loss: 0.0719 - acc: 1.0000
Epoch 5/16
205/205 [==============================] - 1s 4ms/step - loss: 0.0430 - acc: 1.0000
Epoch 6/16
205/205 [==============================] - 1s 4ms/step - loss: 0.0315 - acc: 1.0000
Epoch 7/16
205/205 [==============================] - 1s 4ms/step - loss: 0.0256 - acc: 1.0000
Epoch 8/16
205/205 [==============================] - 1s 4ms/step - loss: 0.0218 - acc: 1.0000
Epoch 9/16
205/205 [==============================] - 1s 4ms/step - loss: 0.0191 - acc: 1.0000
Epoch 10/16
205/205 [==============================] - 1s 4ms/step - loss: 0.0170 - acc: 1.0000
Epoch 11/16
205/205 [===========================

Epoch 11/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0053 - acc: 1.0000
Epoch 12/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0046 - acc: 1.0000
Epoch 13/16
205/205 [==============================] - 2s 7ms/step - loss: 0.0040 - acc: 1.0000
Epoch 14/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0036 - acc: 1.0000
Epoch 15/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0032 - acc: 1.0000
Epoch 16/16
102/102 [==============================] - 4s 34ms/step
[CV]  output_shape=101, nn3=200, nn2=512, nn1=4096, lr=0.001, input_shape=7, decay=1e-09, batch_size=2048, score=0.088, total=  40.9s
[CV] output_shape=101, nn3=200, nn2=2048, nn1=1024, lr=0.01, input_shape=7, decay=1e-06, batch_size=512 
Epoch 1/16
204/204 [==============================] - 9s 45ms/step - loss: 5.3777 - acc: 0.0098
Epoch 2/16
204/204 [==============================] - 0s 2ms/step - loss: 0.4885 - acc: 0.9020
Epoch 3/16
204/204 [==

205/205 [==============================] - 14s 71ms/step - loss: 5.4931 - acc: 0.0098
Epoch 2/16
205/205 [==============================] - 2s 9ms/step - loss: 0.4800 - acc: 0.8976
Epoch 3/16
205/205 [==============================] - 2s 8ms/step - loss: 2.2132 - acc: 0.7805
Epoch 4/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0476 - acc: 0.9854
Epoch 5/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0623 - acc: 0.9805
Epoch 6/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0389 - acc: 0.9902
Epoch 7/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0092 - acc: 0.9951
Epoch 8/16
205/205 [==============================] - 2s 8ms/step - loss: 0.0020 - acc: 1.0000
Epoch 9/16
205/205 [==============================] - 2s 8ms/step - loss: 5.0820e-04 - acc: 1.0000
Epoch 10/16
205/205 [==============================] - 2s 8ms/step - loss: 2.5932e-04 - acc: 1.0000
Epoch 11/16
205/205 [=============================

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 12.6min finished


Epoch 1/16
307/307 [==============================] - 16s 52ms/step - loss: 5.2574 - acc: 0.0065
Epoch 2/16
307/307 [==============================] - 2s 5ms/step - loss: 0.3385 - acc: 0.9967
Epoch 3/16
307/307 [==============================] - 2s 5ms/step - loss: 0.0658 - acc: 1.0000
Epoch 4/16
307/307 [==============================] - 2s 5ms/step - loss: 0.0309 - acc: 1.0000
Epoch 5/16
307/307 [==============================] - 2s 5ms/step - loss: 0.0198 - acc: 1.0000
Epoch 6/16
307/307 [==============================] - 2s 5ms/step - loss: 0.0144 - acc: 1.0000
Epoch 7/16
307/307 [==============================] - 2s 7ms/step - loss: 0.0113 - acc: 1.0000
Epoch 8/16
307/307 [==============================] - 2s 5ms/step - loss: 0.0091 - acc: 1.0000
Epoch 9/16
307/307 [==============================] - 2s 5ms/step - loss: 0.0076 - acc: 1.0000
Epoch 10/16
307/307 [==============================] - 2s 5ms/step - loss: 0.0064 - acc: 1.0000
Epoch 11/16
307/307 [==========================

In [61]:
cv_results_df = pd.DataFrame(grid_result.cv_results_)
cv_results_df.to_csv('gridsearch_VGG16.csv')

In [62]:
cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_output_shape,param_nn3,param_nn2,param_nn1,param_lr,param_input_shape,param_decay,param_batch_size,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,32.193858,0.675317,0.670147,0.076867,101,500,512,4096,0.001,7,1e-06,2048,"{'output_shape': 101, 'nn3': 500, 'nn2': 512, ...",0.194175,0.196078,0.078431,0.156352,0.054969,3
1,12.629730,0.166648,0.787965,0.101065,101,200,2048,1024,0.001,7,1e-09,1024,"{'output_shape': 101, 'nn3': 200, 'nn2': 2048,...",0.145631,0.205882,0.078431,0.143322,0.051973,4
2,21.232271,0.211854,1.140251,0.074460,101,500,2048,2048,0.001,512,0,1024,"{'output_shape': 101, 'nn3': 500, 'nn2': 2048,...",0.155340,0.147059,0.068627,0.123779,0.039049,8
3,13.733493,0.649357,1.433798,0.045567,101,500,512,1024,0.001,7,1e-06,512,"{'output_shape': 101, 'nn3': 500, 'nn2': 512, ...",0.155340,0.137255,0.088235,0.127036,0.028349,7
4,15.167023,0.837273,1.776417,0.202147,101,200,1024,1024,0.0001,512,0,2048,"{'output_shape': 101, 'nn3': 200, 'nn2': 1024,...",0.223301,0.098039,0.078431,0.133550,0.064272,6
5,15.392101,0.376280,2.017972,0.104266,101,500,512,1024,0.0001,7,1e-06,512,"{'output_shape': 101, 'nn3': 500, 'nn2': 512, ...",0.194175,0.156863,0.068627,0.140065,0.052646,5
6,24.241458,0.439833,2.522223,0.129908,101,200,2048,2048,0.0001,7,1e-06,512,"{'output_shape': 101, 'nn3': 200, 'nn2': 2048,...",0.106796,0.117647,0.088235,0.104235,0.012125,9
7,37.399706,0.248150,3.070402,0.320667,101,200,512,4096,0.001,7,1e-09,2048,"{'output_shape': 101, 'nn3': 200, 'nn2': 512, ...",0.194175,0.215686,0.088235,0.166124,0.055640,1
8,18.641577,0.694909,3.087179,0.062845,101,200,2048,1024,0.01,7,1e-06,512,"{'output_shape': 101, 'nn3': 200, 'nn2': 2048,...",0.223301,0.176471,0.088235,0.162866,0.056013,2
9,41.043226,0.227881,3.717691,0.116954,101,1000,1024,4096,0.01,7,0,1024,"{'output_shape': 101, 'nn3': 1000, 'nn2': 1024...",0.058252,0.078431,0.019608,0.052117,0.024369,10


In [63]:
grid_result.best_params_

{'output_shape': 101,
 'nn3': 200,
 'nn2': 512,
 'nn1': 4096,
 'lr': 0.001,
 'input_shape': 7,
 'decay': 1e-09,
 'batch_size': 2048}

In [109]:
# Parameters after RandomizedSearchCV

nn1 = 4096; nn2 = 512; nn3 = 200
lr = 0.001; decay=1e-9
batch_size = 2048

## Regularization

In [110]:
# Regularzation Parameters

dropout = 0.5
l1 = 0.0001
l2 = 0.0001

In [111]:
def model():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    model.add(Dense(NB_CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [112]:
train_labels_onehot = to_categorical(train_labels, NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels, NB_CLASSES)  #One Hot Encoder

In [113]:
top = model()

In [114]:
top.fit(train_data, train_labels_onehot,
              epochs=32,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels_onehot))

Train on 307 samples, validate on 74 samples
Epoch 1/32
307/307 [==============================] - 44s 142ms/step - loss: 81.9862 - acc: 0.1010 - val_loss: 77.5625 - val_acc: 0.8243
Epoch 2/32
307/307 [==============================] - 3s 10ms/step - loss: 78.2338 - acc: 0.5668 - val_loss: 75.0592 - val_acc: 0.8784
Epoch 3/32
307/307 [==============================] - 3s 10ms/step - loss: 75.4331 - acc: 0.7752 - val_loss: 72.4291 - val_acc: 0.8649
Epoch 4/32
307/307 [==============================] - 3s 10ms/step - loss: 72.6787 - acc: 0.8339 - val_loss: 69.6605 - val_acc: 0.9054
Epoch 5/32
307/307 [==============================] - 3s 11ms/step - loss: 69.8011 - acc: 0.9055 - val_loss: 66.8113 - val_acc: 0.9054
Epoch 6/32
307/307 [==============================] - 3s 10ms/step - loss: 66.9261 - acc: 0.9088 - val_loss: 63.9400 - val_acc: 0.9189
Epoch 7/32
307/307 [==============================] - 3s 11ms/step - loss: 64.0132 - acc: 0.9316 - val_loss: 61.0847 - val_acc: 0.9189
Epoch 8/

In [115]:
top.save_weights(top_weights_file)

## Complete Model

In [116]:
def top():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                 
    base= applications.VGG16(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3)) #VGG16 trained on imagenet
    
    top = Sequential()

    top.add(Flatten(input_shape=train_data.shape[1:]))
    top.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())            
    top.add(Dense(NB_CLASSES, activation='softmax'))
    top.load_weights(top_weights_file)
    top.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    
    
    model = Model(input= base.input, output= top(base.output))
    for layer in model.layers:
        layer.trainable = False
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [117]:
model = top()

datagen = ImageDataGenerator()

print("Model Summary")
print(model.summary())

Model Summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0      

In [118]:
generator = datagen.flow_from_directory(
            test_data_dir,
            target_size=(img_width, img_height),
            batch_size=len(test_labels),
            class_mode=None,
            shuffle=False)
test_predictions = model.predict_generator(generator, 1, verbose=1) 

Found 18 images belonging to 11 classes.
1/1 [==============================] - 15s 15s/step


In [119]:
test_predictions = np.asarray(list(map(str,np.argmax(test_predictions,axis=1)))).reshape(-1,1)
test_labels = np.asarray(test_labels).reshape(-1,1)
test_acc, test_acc_op = tf.metrics.accuracy(test_labels, test_predictions)

In [120]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [121]:
test_accuracy = sess.run(test_acc_op)

In [122]:
print('\nTest accuracy : ' + str(test_accuracy))


Test accuracy : 0.9444444


In [123]:
model.save(model_file)